<h3>Combination of reinforcement learning (RL) and heuristic methods to optimize the packing sequence</h3>

<h5>Machine Learning-Based Approach for 2D Bin Packing Problem</h5>
<p>Hybrid Approach: Reinforcement Learning + Heuristic Methods</p>

In [1]:
# import sys
# print(sys.executable)


In [2]:
# pip install gym
# !pip install stable_baselines3
# print(sys.executable)
# !pip uninstall stable_baselines3
# !pip install stable_baselines3
# !pip install stable_baselines3[extra]
# %pip install stable-baselines3


In [3]:
# Import Libraries
# import sys
import numpy as np
import csv
from typing import List, Dict, Tuple
# import gym
# from gym import spaces
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

# Visualization (Optional)
import matplotlib.pyplot as plt
import matplotlib.patches as patches

<h3>2D Bin Packing Environment</h3>

In [4]:
# Define the 2D Bin Packing Environment
class BinPackingEnv(gym.Env):
    def __init__(self, pieces, bin_sizes):  # Change bin_size to bin_sizes
        super(BinPackingEnv, self).__init__()
        self.pieces = pieces
        self.bin_sizes = bin_sizes  # Store all available bin sizes
        self.current_bin_index = 0  # Start with the first bin
        self.bin_size = self.bin_sizes[self.current_bin_index]  # Use one bin at a time
        self.n_pieces = len(pieces)
        max_bin_dimension = max(max(bin_size[0], bin_size[1]) for bin_size in self.bin_sizes)
        self.action_space = spaces.Discrete(self.n_pieces)
        self.observation_space = spaces.Box(
            low=0, 
            # high=np.array([max(bin_size[0], bin_size[1])] * len(self.pieces) * 2).reshape(len(self.pieces), 2), 
            high=np.array([max_bin_dimension] * 2 * len(self.pieces)).reshape(len(self.pieces), 2), 
            shape=(len(self.pieces), 2), 
            dtype=float
        )

        self.reset()

    def switch_to_next_bin(self):
        if self.current_bin_index < len(self.bin_sizes) - 1:
            self.current_bin_index += 1
            self.bin_size = self.bin_sizes[self.current_bin_index]  # Update to the next bin
            print(f"Switched to new bin: {self.bin_size}")
        else:
            print("No more bins available.")

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_bin = np.zeros(self.bin_size)
        self.packed_pieces = []
        # self.state = np.array([[piece[0], piece[1]] for piece in self.pieces]).flatten()
        self.state = np.array([[piece[0], piece[1]] for piece in self.pieces], dtype=np.float32)
        self.reward = 0
        info = {}
        return self.state, info

    def step(self, action):
        piece = self.pieces[action]
        if self.can_place(piece):
            self.place_piece(piece)
            self.packed_pieces.append(piece)
            self.state[action, 0] = piece[0]  # Update length
            self.state[action, 1] = piece[1]  # Update width
            self.reward += 1
        else:
            self.reward -= 1  # Penalty for not placing
    
        done = len(self.packed_pieces) == self.n_pieces
        terminated = done  # Use `terminated` to indicate the end of an episode
        truncated = False  # Use `truncated` to indicate if the episode was truncated
    
        info = {}  # Additional information
    
        return self.state, self.reward, terminated, truncated, info

    def render(self, mode='human'):
        print(f"Packed Pieces: {self.packed_pieces}")

    def can_place(self, piece):
        # Simple heuristic to check if the piece can fit in the current bin
        return piece[0] <= self.bin_size[0] and piece[1] <= self.bin_size[1]

    # CoPilot
    def can_place(self, piece):
        # Simple heuristic to check if the piece can fit in the current bin
        return piece[0] <= self.bin_size[0] and piece[1] <= self.bin_size[1]

    def place_piece(self, piece):
        # Place the piece in the bin (simplified for illustration)
        self.current_bin[:piece[0], :piece[1]] = 1


In [5]:
gap = 0  # Gap between parts in mm

<h3>Define file paths</h3>

In [6]:
# Define file paths
glass_data_file = 'data/glass_data.csv'

# Load Data Functions
def load_glass_data(filepath: str) -> List[Dict]:
    with open(filepath, 'r') as file:
        reader = csv.DictReader(file)
        return [{'location': row['location'], 
                 'length': int(row['glass_length']), 
                 'height': int(row['glass_height']), 
                 'qty': int(row['glass_qty'])} for row in reader]

# Convert dictionary data to tuple and expand based on quantity
def pieces_dict_to_expanded_tuples(data: List[Dict]) -> List[Tuple]:
    pieces_expanded_tuples = []
    for item in data:
        for _ in range(item['qty']):
            pieces_expanded_tuples.append((item['length'], item['height']))
    return pieces_expanded_tuples

def calculate_total_area(pieces: List[Tuple[int, int]]) -> int:
    return sum((length / 1000) * (height / 1000) for length, height in pieces)

# Load the data
pieces = load_glass_data(glass_data_file)

# Convert to expanded tuples
pieces = pieces_dict_to_expanded_tuples(pieces)  # Changed variable name to pieces

# Calculate the total area of all pieces
total_area = calculate_total_area(pieces)

# Print the total area
print(f"Total area of all Glass: {total_area:.3f} Sqm")

# Add this line after pieces is created
print(f"Number of Glass: {len(pieces)}")

# Simply print the pieces list
# print(pieces)

Total area of all Glass: 451.142 Sqm
Number of Glass: 260


In [7]:
# # Define file paths
# glass_data_file = 'data/glass_data.csv'

# # Load Data Functions
# def load_glass_data(filepath: str) -> List[Dict]:
#     with open(filepath, 'r') as file:
#         reader = csv.DictReader(file)
#         return [{'location': row['location'], 
#                  'length': int(row['glass_length']), 
#                  'height': int(row['glass_height']), 
#                  'qty': int(row['glass_qty'])} for row in reader]

# # Convert dictionary data to tuple and expand based on quantity
# def pieces_dict_to_expanded_tuples(data: List[Dict]) -> List[Tuple]:
#     pieces_expanded_tuples = []
#     for item in data:
#         for _ in range(item['qty']):
#             pieces_expanded_tuples.append((item['length'], item['height']))
#     return pieces_expanded_tuples

# # Load the data
# pieces = load_glass_data(glass_data_file)

# # Convert to expanded tuples
# pieces_expanded_tuples = pieces_dict_to_expanded_tuples(pieces)

# # Print the count of pieces based on quantity
# print(f"Total number of pieces: {len(pieces_expanded_tuples)}")

# # Print the expanded tuples
# for t in pieces_expanded_tuples:
#     print(t)


In [8]:
# Define file paths
stock_sizes_file = 'data/glass_sheet_size1.csv'

# Load Data Functions
def load_stock_sizes(filepath: str) -> List[Dict]:
    with open(filepath, 'r') as file:
        reader = csv.DictReader(file)
        return [{'length': int(row['length']), 
                 'width': int(row['width']), 
                 'qty': int(row['qty'])} for row in reader]

# define the get_bin_sizes function
def get_bin_sizes(data: List[Dict]) -> List[Tuple[int, int]]:
    return [(item['length'], item['width']) for item in data]

def bins_dict_to_tuples(data: List[Dict]) -> List[Tuple]:
    bins_expanded_tuples = []
    for item in data:
        for _ in range(item['qty']):
            bins_expanded_tuples.append((item['length'], item['width']))
    return bins_expanded_tuples

def calculate_total_area(bins: List[Tuple[int, int]]) -> int:
    return sum((length / 1000) * (width / 1000) for length, width in bins)

# Load the data
bins = load_stock_sizes(stock_sizes_file)

# Convert to expanded tuples
bins = bins_dict_to_tuples(bins)  # Changed variable name to bins

# Calculate the total area of all bins
total_area = calculate_total_area(bins)

# Print the total area
print(f"Total area of all bins: {total_area:.3f} Sqm")

# Add this line after bins is created
print(f"Number of bins: {len(bins)}")

# Simply print the bins list
# print(bins)

Total area of all bins: 2009.520 Sqm
Number of bins: 300


In [9]:
# # Define file paths
# stock_sizes_file = 'data/glass_sheet_size1.csv'

# # Load Data Functions
# def load_stock_sizes(filepath: str) -> List[Dict]:
#     with open(filepath, 'r') as file:
#         reader = csv.DictReader(file)
#         return [{'length': int(row['length']), 
#                  'width': int(row['width']), 
#                  'qty': int(row['qty'])} for row in reader]

# # Convert dictionary data to tuple
# def get_bin_sizes(data: List[Dict]) -> List[Tuple]:
#     # Convert all bin sizes to tuples
#     return [(item['length'], item['width']) for item in data]

# def calculate_total_quantity(bin_sizes: List[Dict]) -> int:
#     return sum(item['qty'] for item in bin_sizes)

# # Load the data
# bins_data = load_stock_sizes(stock_sizes_file)

# # Get all bin sizes
# bin_size = get_bin_sizes(bins_data)

# # Calculate the total quantity of all bin sizes
# total_quantity = calculate_total_quantity(bins_data)

# # Print count and bin sizes
# print(f"Number of bin sizes: {len(bin_size)}")
# print(bin_size)

In [10]:
# # Define file paths
# stock_sizes_file = 'data/glass_sheet_size1.csv'

# # Load Data Functions
# def load_stock_sizes(filepath: str) -> List[Dict]:
#     with open(filepath, 'r') as file:
#         reader = csv.DictReader(file)
#         return [{'length': int(row['length']), 
#                  'width': int(row['width']), 
#                  'qty': int(row['qty'])} for row in reader]

# # Convert dictionary data to tuple and expand based on quantity
# def bins_dict_to_expanded_tuples(data: List[Dict]) -> List[Tuple]:
#     bins_expanded_tuples = []
#     for item in data:
#         for _ in range(item['qty']):
#             bins_expanded_tuples.append((item['length'], item['width']))
#     return bins_expanded_tuples

# # Load the data
# bin_size = load_stock_sizes(stock_sizes_file)

# # Convert to expanded tuples
# bins_expanded_tuples = bins_dict_to_expanded_tuples(bin_size)

# # Print the count of pieces based on quantity
# print(f"Total number of bins: {len(bins_expanded_tuples)}")

# # Print the expanded tuples
# for b in bins_expanded_tuples:
#     print(b)

<h3>Heuristic Placement Strategy (Best Fit Decreasing)</h3>

In [11]:
def best_fit_decreasing(pieces, bin_size):
    bins = []
    for piece in sorted(pieces, key=lambda x: max(x), reverse=True):
        placed = False
        for bin in bins:
            if bin['remaining_width'] >= piece[0] and bin['remaining_height'] >= piece[1]:
                bin['placements'].append(piece)
                bin['remaining_width'] -= piece[0]
                bin['remaining_height'] -= piece[1]
                placed = True
                break
        if not placed:
            new_bin = {
                'size': bin_size,
                'remaining_width': bin_size[0] - piece[0],
                'remaining_height': bin_size[1] - piece[1],
                'placements': [piece]
            }
            bins.append(new_bin)
    return bins


In [12]:
# # Heuristic Placement Strategy (Best Fit Decreasing)
# def best_fit_decreasing(pieces, bin_size):
#     pieces.sort(key=lambda x: x[0] * x[1], reverse=True)
#     bins = []
#     for piece in pieces:
#         placed = False
#         for bin in bins:
#             if bin['remaining_width'] >= piece[0] and bin['remaining_height'] >= piece[1]:
#                 bin['placements'].append(piece)
#                 bin['remaining_width'] -= piece[0]
#                 placed = True
#                 break
#         if not placed:
#             new_bin = {
#                 'size': bin_size,
#                 'remaining_width': bin_size[0],
#                 'remaining_height': bin_size[1],
#                 'placements': [piece]
#             }
#             bins.append(new_bin)
#     return bins

<h3>Create a sample dataset of pieces and bin size</h3>

In [13]:
# Create a sample dataset of pieces and bin size
pieces_raw = load_glass_data(glass_data_file)
bin_size_raw = load_stock_sizes(stock_sizes_file)

# Convert pieces to expanded tuples
pieces = pieces_dict_to_expanded_tuples(pieces_raw)

# Convert bin_size to tuples
bin_size = get_bin_sizes(bin_size_raw)

# Initialize the environment
env = BinPackingEnv(pieces, bin_size)
check_env(env)

D:\Python311\Lib\site-packages\stable_baselines3\common\env_checker.py:272: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


In [14]:
# Train the model
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 327      |
|    ep_rew_mean     | 3.1e+04  |
| time/              |          |
|    fps             | 649      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 327           |
|    ep_rew_mean          | 3.14e+04      |
| time/                   |               |
|    fps                  | 544           |
|    iterations           | 2             |
|    time_elapsed         | 7             |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.00016927425 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2       

In [15]:
# Test the trained model
obs, info = env.reset()  # Get the observation and info
for _ in range(len(pieces)):
    action, _states = model.predict(obs)
    obs, rewards, terminated, truncated, info = env.step(action)
    env.render()
    if terminated or truncated:
        break


Packed Pieces: [(952, 1738)]
Packed Pieces: [(952, 1738), (973, 1608)]
Packed Pieces: [(952, 1738), (973, 1608), (964, 1603)]
Packed Pieces: [(952, 1738), (973, 1608), (964, 1603), (964, 1603)]
Packed Pieces: [(952, 1738), (973, 1608), (964, 1603), (964, 1603), (1010, 1594)]
Packed Pieces: [(952, 1738), (973, 1608), (964, 1603), (964, 1603), (1010, 1594), (1010, 1594)]
Packed Pieces: [(952, 1738), (973, 1608), (964, 1603), (964, 1603), (1010, 1594), (1010, 1594)]
Packed Pieces: [(952, 1738), (973, 1608), (964, 1603), (964, 1603), (1010, 1594), (1010, 1594), (964, 1902)]
Packed Pieces: [(952, 1738), (973, 1608), (964, 1603), (964, 1603), (1010, 1594), (1010, 1594), (964, 1902), (964, 1902)]
Packed Pieces: [(952, 1738), (973, 1608), (964, 1603), (964, 1603), (1010, 1594), (1010, 1594), (964, 1902), (964, 1902), (1010, 1594)]
Packed Pieces: [(952, 1738), (973, 1608), (964, 1603), (964, 1603), (1010, 1594), (1010, 1594), (964, 1902), (964, 1902), (1010, 1594), (973, 1608)]
Packed Pieces: [

<h3>Apply heuristic placement strategy</h3>

In [16]:
# Apply heuristic placement strategy
bins = best_fit_decreasing(pieces, bin_size)
print("Bins after heuristic placement:")
for i, bin in enumerate(bins):
    print(f"Bin {i+1}: {bin['placements']}")

def plot_bin(bin):
    fig, ax = plt.subplots()
    for piece in bin['placements']:
        rect = patches.Rectangle((0, 0), piece[0], piece[1], linewidth=1, edgecolor='r', facecolor='b')
        ax.add_patch(rect)
    plt.xlim(0, bin['size'][0])
    plt.ylim(0, bin['size'][1])
    plt.show()

for bin in bins:
    plot_bin(bin)

TypeError: unsupported operand type(s) for -: 'tuple' and 'int'